In [1]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
import numpy
 
sys.path.append(".")

import config

In [ ]:
print(config)

In [2]:
PROJECT_PATH = config.data_directory

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
# try to not get banned
import random
import time

USER_AGENTS = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:11.0) Gecko/20100101 Firefox/11.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36 OPR/58.0.3135.107']

def getLyric(title, user_agent=True):
    url = "https://www.azlyrics.com/lyrics/{}.html".format(title)
    time.sleep(random.randint(1,10))
    if user_agent:
        return requests.get(url, headers={'User-Agent': random.choice(USER_AGENTS)})
    return requests.get(url)

In [5]:
def create_file(title):
    file_path = '{}/{}.txt'.format(PROJECT_PATH, title)
    if not os.path.exists(file_path):
        result = getLyric(title)
        c = result.content
        soup = BeautifulSoup(c)
        ringtones = soup.find_all("div", class_="ringtone")
        
        if len(ringtones) > 0:
            ringtone = ringtones[0]
            html_lyrics = ringtone.find_next_sibling('div')
            cleanText = html_lyrics.get_text().replace('\r','').replace('\n',' ').replace("\'",'"')

            file_path = '{}/{}.txt'.format(PROJECT_PATH, title.replace('/','-'))

            fo = open(file_path, "w", encoding='utf-8')
            fo.write(cleanText)
            fo.close()
            return 1
        else:
            return 0
    else:
        print('File Already Exists')

In [6]:
create_file('wizkhalifa/youngwildfree')

1

In [10]:
getLyric('wizkhalifa/youngwildfree')

<Response [200]>

In [11]:
def create_songs_file(name):
    csvData = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_release_date_precision', 'album_total_tracks']]

    with open('{}/{}.csv'.format(PROJECT_PATH, name), 'w', newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(csvData)

    csvFile.close()

In [ ]:
create_songs_file('songs')

In [12]:
def fetch_songs(offset, count, name = 'songs', limit = 50):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    new_file = 'a'
    tracks = []
    if not os.path.exists(file_path):
        tracks = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_release_date_precision', 'album_total_tracks']]
        new_file = 'w'
        
    with open(file_path, new_file, newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        for i in range(offset, count * limit, limit):
            track_results = sp.search(q='year:1990', type='track', limit=limit,offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                tracks.append([t['id'], t['name'], t['popularity'], t['duration_ms'], t['explicit'], t['artists'][0]['name'], t['artists'][0]['id'],  t['album']['name'], t['album']['id'], t['album']['release_date'], t['album']['release_date_precision'], t['album']['total_tracks']])
        writer.writerows(tracks)
    csvFile.close()


In [13]:
fetch_songs(0, 10)

In [14]:
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

In [15]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67
2,6m59VvDUi0UQsB2eZ9wVbH,Poison,69,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10
3,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19


In [16]:
pattern = r'[^a-z0-9A-z]'
df['name_az'] = df['name'].str.replace(pattern, '').str.lower()
df['artist_az'] = df['artist'].str.replace(pattern, '').str.lower()
df['has_lyrics'] = 0

In [18]:
for index, row in df.iterrows():
    print(row['name_az'], row['artist_az'])
    df.loc[index, 'has_lyrics'] = create_file('{}/{}'.format(row['artist_az'], row['name_az']))

thunderstruck acdc
dontgobreakingmyheart eltonjohn
poison bellbivdevoe
maninthebox aliceinchains
istanbul theymightbegiants
canikickit atribecalledquest
morethanwords extreme
whatilikeaboutyou theromantics
cowboysfromhell pantera
ucanttouchthis mchammer
stayinalivefromsaturdaynightfeversoundtrack beegees
thehumptydance digitalunderground
cherrypie warrant
holdon wilsonphillips
hardtohandle theblackcrowes
johndeeregreen joediffie
thereshegoes thelas
enjoythesilencesinglemix depechemode
windofchange scorpions
pickupman joediffie
unchainedmelody therighteousbrothers
blackbetty ramjam
nothingcompares2u sinadoconnor
hungerstrike templeofthedog
dontcloseyoureyes keithwhitley
abrzamemuyfuerte juangabriel
cemeterygates pantera
rayandoelsol man
mamasaidknockyouout llcoolj
propmeupbesidethejukeboxifidie joediffie
holywarsthepunishmentdueremastered2004 megadeth
pickupman joediffie
iceicebaby vanillaice
feelsgood tonytoniton
howdeepisyourlovefromsaturdaynightfeversoundtrack beegees
iliketheway hif

sadsongssaysomuch eltonjohn
stompinatthesavoy bennygoodman
meaculpa enigma
sitdown james
skiesthelimit fleetwoodmac
givinyourselfaway ratt
fromadistance bettemidler
lamer charlestrenet
stop janesaddiction
hazyshadeofwinter thebangles
heroessingleversion1990remasteredversion davidbowie
crazyforyoueditversion madonna
trenalsur losprisioneros
electricboogie marciagriffiths
weareinlove harryconnickjr
allshewrotelive firehouse
uncletomscabin warrant
brotherjukebox markchesnutt
icedance dannyelfman
getthefunkout extreme
crownofthorns motherlovebone
candy iggypop
youreinlove wilsonphillips
thesleep pantera
louderthanlove tka
isawred warrant
lovecanbuildabridge thejudds
somethingslastalongtime danieljohnston
freaksoftheindustry digitalunderground
amerikkkasmostwanted icecube
lieutenantkijop60iiromance sergeiprokofiev
seeingthings theblackcrowes
holehearted extreme
skeletonsofsociety slayer
fiftyfiftyclown cocteautwins
devilsplaything danzig
allnightthing templeofthedog
signsliveatthetrocadero1

In [21]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,name_az,artist_az,has_lyrics
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12,thunderstruck,acdc,1
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67,dontgobreakingmyheart,eltonjohn,1
2,6m59VvDUi0UQsB2eZ9wVbH,Poison,69,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10,poison,bellbivdevoe,1
3,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12,maninthebox,aliceinchains,1
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19,istanbul,theymightbegiants,0


In [22]:
df.groupby('has_lyrics').count()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,name_az,artist_az
has_lyrics,,,,,,,,,,,,,,
0,227,227,227,227,227,227,227,227,227,227,227,227,227,227
1,273,273,273,273,273,273,273,273,273,273,273,273,273,273


In [ ]:
import spacy
from spacy_cld import LanguageDetector
 
nlp = spacy.load('en')
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)
doc = nlp('This is some English text.')
 
doc._.languages  # ['en']
doc._.language_scores['en']  # 0.96